#Installations and imports

In [ ]:
!pip install yfinance

In [ ]:
!pip install plotly

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import datetime
import yfinance as yf
import plotly.express as plt

# Code

## Extracting data for Portfolio stocks - 5 year

In [ ]:
# Preparing data
stocks_dict = {'DEEPAKFERT.NS':14797.5, 'HDFCBANK.NS' : 83127, 'SBIN.NS':16023, 'KEI.NS':0, 'ICICIBANK.NS':19186, 'CROMPTON.NS':20062,
               'ITC.NS':12663, 'POLYPLEX.NS':20184, 'VBL.NS':0, 'AEGISCHEM.NS':0, 'CGPOWER.NS':0,
               'KEC.NS':0, 'BALKRISIND.NS':0, 'AMBUJACEM.NS':0, 'DMART.NS':0, 'UPL.NS':0, 'IPCALAB.NS':0, 'NHPC.NS':0}
end = datetime.date.today()
start = end - datetime.timedelta(days = 5*365)

stocks_df=pd.DataFrame(list(stocks_dict.items()), columns=['TICKER','AMOUNT_INVESTED'])
stocks_df['PROPORTION']=stocks_df['AMOUNT_INVESTED']/stocks_df['AMOUNT_INVESTED'].sum()

## Building dataframe


In [ ]:
stocks_df.sort_values(by='AMOUNT_INVESTED', ascending=False, inplace=True)
stocks_list=stocks_df['TICKER'].tolist()

In [ ]:
# Extracting data
data = yf.download(stocks_list, start=start, end=end, group_by='ticker')

[*********************100%***********************]  18 of 18 completed


In [ ]:
# Filtering dataframe based on single column - Adj Close
filter_list = []
for stock in stocks_list:
  filter_list.append((stock,'Adj Close'))
filtered_data = data.filter(filter_list)

In [ ]:
filtered_data.shape[0]
# filtered_data.head(5)

1231

## Calculating returns & risk for each stock

In [ ]:
returns_df = filtered_data.pct_change(1).dropna()

In [ ]:
returns_df.shape[0]

1230

In [ ]:
individual_stocks = np.std(returns_df) * np.sqrt(250)

## Calculating risk of entire portfolio

In [ ]:
# Preparing weights list
weights = stocks_df['PROPORTION'].tolist()

In [ ]:
def getPortRisk(weights):
  cov_matrix = returns_df.cov()
  variance_portfolio = np.dot(np.transpose(weights), np.dot(cov_matrix, weights))
  annual_risk_portfolio = np.sqrt(variance_portfolio) * np.sqrt(250)
  return annual_risk_portfolio

## Minimizing risk by optimizing weights


In [ ]:
bounds = tuple((0,1) for i in range(len(stocks_list)))

In [ ]:
cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1})

In [ ]:
results = minimize(fun=getPortRisk, x0=weights, bounds=bounds, constraints=cons)

## Optimizing weights

In [ ]:
total_portfolio_amount=stocks_df['AMOUNT_INVESTED'].sum()
print("Total portfolio amount: ",total_portfolio_amount)

optimized_weights=pd.DataFrame(results['x'])
optimized_weights.index=filtered_data.columns #Indexing based on stocknames
optimized_weights.rename(columns={0:"WEIGHTS"}, inplace=True) #Renaming

#------------- Adding 3 columns - Amount_SHOULD_INVEST, AMOUNT_INVESTED, DIFFERENCE --------------
optimized_weights['AMOUNT_SHOULD_INVEST']=round(optimized_weights['WEIGHTS']*total_portfolio_amount,0)
optimized_weights['AMOUNT_INVESTED']=stocks_df['AMOUNT_INVESTED'].tolist()
optimized_weights['DIFFERENCE']=round(optimized_weights['AMOUNT_SHOULD_INVEST']-optimized_weights['AMOUNT_INVESTED'],2)
#-------------------------------------------------------------------------------------------------

optimized_weights['WEIGHTS'] = optimized_weights['WEIGHTS'].apply(lambda x:round(x,4)) # rounding weights

Total portfolio amount:  186042.5


In [ ]:
# Risk AFTER weight optimization
modified_weights=optimized_weights['WEIGHTS'].tolist()
getPortRisk(modified_weights)

0.16240742176673362

In [ ]:
# Risk BEFORE weight optimization
getPortRisk(weights)

0.22126555826871397

##Visualization

In [ ]:
individual_risk_df = pd.DataFrame(individual_stocks, columns=['Individual_Risk'])
graph_df = pd.merge(optimized_weights, individual_risk_df, left_index=True, right_index=True)
graph_df.reset_index(inplace=True)

In [ ]:
graph_df.rename(columns={'level_0':'TICKER'}, inplace=True)
graph_df.drop(['level_1'], axis=1, inplace=True)

In [ ]:
graph_df.sort_values(by='WEIGHTS', ascending=False, inplace=True)
graph_df

,TICKER,WEIGHTS,AMOUNT_SHOULD_INVEST,AMOUNT_INVESTED,DIFFERENCE,Individual_Risk
8,IPCALAB.NS,0.1842,34267.0,0.0,34267.0,0.309569
6,ITC.NS,0.1698,31596.0,12663.0,18933.0,0.264618
0,HDFCBANK.NS,0.1580,29389.0,83127.0,-53738.0,0.268750
17,NHPC.NS,0.1327,24688.0,0.0,24688.0,0.317746
15,VBL.NS,0.0962,17893.0,0.0,17893.0,0.380224
2,CROMPTON.NS,0.0801,14906.0,20062.0,-5156.0,0.322400
10,DMART.NS,0.0793,14759.0,0.0,14759.0,0.331438
14,CGPOWER.NS,0.0319,5944.0,0.0,5944.0,0.518309
13,KEC.NS,0.0248,4607.0,0.0,4607.0,0.377212
16,KEI.NS,0.0146,2712.0,0.0,2712.0,0.442481


In [ ]:
graph_df.to_csv('Optimize weights.csv')

In [ ]:
fig = plt.scatter(graph_df ,x='Individual_Risk', y='WEIGHTS', text='TICKER')
fig.update_traces(textposition='top center')

fig.update_layout(
    height=600,
    width=900,
    title_text='Risk and Weight allocated for each stock'
)
fig.show()